In [28]:
import numpy as np
from tkinter import *
from tkinter import ttk
import tkinter.scrolledtext as tkst
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.animation import FuncAnimation
import math
import pandas as pd
marathon_2015_2017 = pd.read_csv("./marathon_2015_2017.csv")

In [29]:
marathon_2015_2017['M/F'] = marathon_2015_2017['M/F'].map({'M': 1, 'F': 0})
# 2015-2016 데이터와  2017 데이터로 분리
marathon_2015_2016 = marathon_2015_2017[marathon_2015_2017['Year'] != 2017]
marathon_2017 = marathon_2015_2017[marathon_2015_2017['Year'] == 2017]

# 별도의 데이터 프레임으로 필요한 필드만 취하여 저장
df_2015_2016 = pd.DataFrame(marathon_2015_2016, \
        columns=['M/F',  'Age',  'Pace',  '10K', '20K',  '30K',  'Official Time'])
df_2017 = pd.DataFrame(marathon_2017, \
        columns=['M/F',  'Age',  'Pace',  '10K', '20K',  '30K',  'Official Time'])

# 각 데이터 프레임을 리스트로 변환
record_2015_2016 = df_2015_2016.values.tolist()
record_2017 = df_2017.values.tolist()

# train 과 test   X와 y로 변환
x_train = [ r[0:3] for r in record_2015_2016]
y_train = [ r[3:7] for r in record_2015_2016]
x_test = [ r[0:3] for r in record_2017]
y_test = [ r[3:7] for r in record_2017]

gender_list = ['Female', 'Male']
fig = Figure(figsize=(10,4), dpi=100)
ax = fig.add_subplot(111)
ax.set_xlim(15, 88)  # 나이를 표시할 x축의 범위
ax.set_ylim(0, 1300) # 페이스가 표시될 y축의 범위
ax.set_ylabel("Pace : Runner's overall minute per mile pace")
ax.set_xlabel("Age : Age on race day")
xdata, ydata = [], []

In [30]:
def histogram():
    t_a = int(t_aSpbox.get()) - 1  # 순위 데이터는 0부터 1위로 표시
    runner = x_test[t_a]
    t_g = int(runner[0])
    t_a = int(runner[1])
    t_p = int(runner[2])
    if(t_g): gender_color = 'b'
    else: gender_color = 'r'
    gender_record = df_2017[df_2017['M/F'] == t_g]
    gender_age_record = gender_record[gender_record.Age == t_a] 
    gender_age_record_list = gender_age_record.values.tolist() 
    ax.plot(gender_record.Age, gender_record.Pace, '.', color=gender_color, alpha=0.3)
    ax.plot(t_a, t_p, 'yd')
    stat = gender_age_record['Pace'].describe()
    title = 'Gender : '+gender_list[t_g]+', Age : '+str(t_a)+', Pace : '+str(t_p)
    ax.set_title(title) 
    ax.annotate('['+gender_list[t_g]+', '+str(t_a)+']', (75, 1200), fontsize=10)
    ax.annotate("%10s %7i" % ('Count : ', stat[0]), (75, 1150), fontsize=10)
    ax.annotate("%10s %7.3f" % ('Mean :  ', stat[1]), (75, 1100), fontsize=10)
    ax.annotate("%10s %7.3f" % ('25% :   ', stat[3]), (75, 1050), fontsize=10)
    ax.annotate("%10s %7.3f" % ('75% :   ', stat[5]), (75, 1000), fontsize=10)
    fig.canvas.draw() 

In [31]:
def seconds_to_hhmmss(seconds):
    hours = seconds // (60*60)
    seconds %= (60*60)
    minutes = seconds // 60
    seconds %= 60
    return "%02i:%02i:%02i" % (hours, minutes, seconds)

In [32]:
def learning():
    import tensorflow as tf
    import numpy as np
    # 입력한 순위의 참가자의 성별 나이 페이스를 각 변수에 저장
    t_a = int(t_aSpbox.get()) - 1
    runner = x_test[t_a]  # 예측 및 테스트 데이터 x_test -  2017년 데이터
    t_g = int(runner[0]) # 성별
    t_y = int(runner[1]) # 나이
    t_p = int(runner[2]) # 페이스
    t_t = int(t_tSpbox.get())  
    t_r = float(t_rSpbox.get())
    
    model =  tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(4, input_shape=(3,)))  # 출력 갯수 4
    sgd = tf.keras.optimizers.SGD(lr=t_r)
    model.compile(loss='mse', optimizer=sgd)
    model.summary()
    # x_train , y_train 학습  - 2015-2016년 데이터년 데이터
    history = model.fit(np.array(x_train), np.array(y_train), epochs=t_t)
    
    log_ScrolledText.insert(END,'\nGender:'+gender_list[t_g]+',Age:'+str(t_y)+\
                                            ',Pace :'+str(t_p)+'\n', 'TITLE')
    log_ScrolledText.insert(END,'\n\nCost Decent\n\n','HEADER')
    log_ScrolledText.insert(END,"%20s %20s" % ('Step', 'Cost')+'\n\n')
    for step in range(t_t):
        if step % 100 == 0:
            cost_val = history.history['loss'][step]
            log_ScrolledText.insert(END, "%20i %20.5f" % (step, cost_val)+'\n')   
    winner = [ t_g, t_y, t_p ]
    time = model.predict(np.array([winner]))
    log_ScrolledText.insert(END, "%20s" % ('\n\nRecords Prediction\n\n'), 'HEADER')
    log_ScrolledText.insert(END, "%20s %30s %30s %20s" % ('Distance(km)', \
                        'Real record', 'ML Prediction', 'Variation(Second)')+'\n\n')
    distance = [ 10., 20., 30., 42.195 ]
    for i in range(len(time[0])): # 예측 결과 갯수 만큼 반복
        dist_time = int(time[0][i])
        real_time = seconds_to_hhmmss(y_test[t_a][i]) + '(' + str(y_test[t_a][i]) + ')'
        ml_time = seconds_to_hhmmss(dist_time) + '(' + str(dist_time) + ')'
        variation = y_test[t_a][i] - dist_time
        log_ScrolledText.insert(END, "%20.3f %30s %30s %20.3f" % (distance[i], \
                                            real_time, ml_time, variation)+'\n') 

In [33]:
#윈도우 생성
main = Tk()
main.title("Multi Variable Output Linear Regression")
main.geometry()
# 윈도우 제목 라벨
label=Label(main, text='Multi Variable Output Linear Regression')
label.config(font=("굴림", 18))
label.grid(row=0,column=0,columnspan=6)
# 순위를 지정할 스핀 박스 & 라벨
t_aVal  = IntVar(value=1)
t_aSpbox = Spinbox(main, textvariable=t_aVal ,from_=0, to=len(x_test), increment=1 )
t_aSpbox.grid(row=1,column=1)
t_aLabel=Label(main, text='Rank of runner : ')                
t_aLabel.grid(row=1,column=0)
# 학습 횟수를 지정할 스핀박스 & 라벨
t_tVal  = IntVar(value=2000)
t_tSpbox = Spinbox(main, textvariable=t_tVal ,from_=0, to=100000, increment=1000 )
t_tSpbox.grid(row=1,column=3)
t_tLabel=Label(main, text='Number of train : ')                
t_tLabel.grid(row=1,column=2)
#러닝 레이트를 지정할 스핀박스 & 라벨
t_rVal  = DoubleVar(value=1e-6)
t_rSpbox = Spinbox(main, textvariable=t_rVal ,from_=0, to=1, increment=1e-6 )
t_rSpbox.grid(row=1,column=5)
t_rLabel=Label(main, text='Learning rate : ')                
t_rLabel.grid(row=1,column=4)
# 그래프 그리는 버튼과 예측버튼 
btn1 = Button(main,text="Histogram", height=2,command=lambda:histogram())
btn1.grid(row=2, column=0, columnspan=3, sticky=(W, E))
btn2 = Button(main,text="Prediction", height=2,command=lambda:learning())
btn2.grid(row=2, column=3, columnspan=3, sticky=(W, E))
# 차트 영역
grad_canvas = FigureCanvasTkAgg(fig, main)
grad_canvas.get_tk_widget().grid(row=3,column=0,columnspan=6)
# 스크롤드텍스트 와 스타일 설정
log_ScrolledText = tkst.ScrolledText(main, height=15)
log_ScrolledText.grid(row=4,column=0,columnspan=6, sticky=(N, S, W, E))
log_ScrolledText.configure(font='굴림')
log_ScrolledText.tag_config('RESULT', foreground='blue', font=("굴림", 12))
log_ScrolledText.tag_config('HEADER', foreground='black', font=("굴림", 14))
log_ScrolledText.tag_config('TITLE', foreground='red', font=("굴림", 18))

In [34]:
main.mainloop()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 16        
Total params: 16
Trainable params: 16
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2000
1664/1664 [==============================] - 1s 480us/step - loss: 1375532.8210
Epoch 2/2000
1664/1664 [==============================] - 1s 474us/step - loss: 144039.1088
Epoch 3/2000
1664/1664 [==============================] - 1s 477us/step - loss: 114108.9504
Epoch 4/2000
1664/1664 [==============================] - 1s 473us/step - loss: 106407.9938
Epoch 5/2000
1664/1664 [==============================] - 1s 482us/step - loss: 110429.8151
Epoch 6/2000
1664/1664 [==============================] - 1s 470us/step - loss: 102508.5889
Epoch 7/2000
1664/1664 [==============================] - 1s 482us/step - loss: 113244.5702
Epoch 8/2

1664/1664 [==============================] - 1s 472us/step - loss: 119709.2979
Epoch 85/2000
1664/1664 [==============================] - 1s 469us/step - loss: 110791.6522
Epoch 86/2000
1664/1664 [==============================] - 1s 473us/step - loss: 109834.1790
Epoch 87/2000
1664/1664 [==============================] - 1s 481us/step - loss: 117341.7886
Epoch 88/2000
1664/1664 [==============================] - 1s 472us/step - loss: 110257.6274
Epoch 89/2000
1664/1664 [==============================] - 1s 471us/step - loss: 111910.6826
Epoch 90/2000
1664/1664 [==============================] - 1s 471us/step - loss: 114346.0376
Epoch 91/2000
1664/1664 [==============================] - 1s 481us/step - loss: 102981.3160
Epoch 92/2000
1664/1664 [==============================] - 1s 471us/step - loss: 107317.4476
Epoch 93/2000
1664/1664 [==============================] - 1s 480us/step - loss: 127069.8144
Epoch 94/2000
1664/1664 [==============================] - 1s 471us/step - loss: 120

1664/1664 [==============================] - 1s 481us/step - loss: 106069.6240
Epoch 172/2000
1664/1664 [==============================] - 1s 481us/step - loss: 105815.0293
Epoch 173/2000
1664/1664 [==============================] - 1s 478us/step - loss: 108327.9356
Epoch 174/2000
1664/1664 [==============================] - 1s 487us/step - loss: 109002.6756
Epoch 175/2000
1664/1664 [==============================] - 1s 475us/step - loss: 138814.6336
Epoch 176/2000
1664/1664 [==============================] - 1s 474us/step - loss: 106151.7220
Epoch 177/2000
1664/1664 [==============================] - 1s 479us/step - loss: 110618.5667
Epoch 178/2000
1664/1664 [==============================] - 1s 491us/step - loss: 109755.2365
Epoch 179/2000
1664/1664 [==============================] - 1s 491us/step - loss: 110857.2786
Epoch 180/2000
1664/1664 [==============================] - 1s 473us/step - loss: 107937.4481
Epoch 181/2000
1664/1664 [==============================] - 1s 482us/step -

1664/1664 [==============================] - 1s 476us/step - loss: 117511.3678
Epoch 259/2000
1664/1664 [==============================] - 1s 480us/step - loss: 101532.7594
Epoch 260/2000
1664/1664 [==============================] - 1s 479us/step - loss: 105419.8319
Epoch 261/2000
1664/1664 [==============================] - 1s 475us/step - loss: 107263.3832
Epoch 262/2000
1664/1664 [==============================] - 1s 479us/step - loss: 106843.8853
Epoch 263/2000
1664/1664 [==============================] - 1s 472us/step - loss: 130719.9004
Epoch 264/2000
1664/1664 [==============================] - 1s 477us/step - loss: 98541.0938
Epoch 265/2000
1664/1664 [==============================] - 1s 472us/step - loss: 109770.9579
Epoch 266/2000
1664/1664 [==============================] - 1s 472us/step - loss: 119675.0143
Epoch 267/2000
1664/1664 [==============================] - 1s 472us/step - loss: 104311.3107
Epoch 268/2000
1664/1664 [==============================] - 1s 466us/step - 

1664/1664 [==============================] - 1s 474us/step - loss: 109052.0571
Epoch 346/2000
1664/1664 [==============================] - 1s 468us/step - loss: 105243.7824
Epoch 347/2000
1664/1664 [==============================] - 1s 480us/step - loss: 104354.5473
Epoch 348/2000
1664/1664 [==============================] - 1s 472us/step - loss: 107573.8982
Epoch 349/2000
1664/1664 [==============================] - 1s 479us/step - loss: 100058.4591
Epoch 350/2000
1664/1664 [==============================] - 1s 471us/step - loss: 105384.1515
Epoch 351/2000
1664/1664 [==============================] - 1s 479us/step - loss: 112841.7224
Epoch 352/2000
1664/1664 [==============================] - 1s 475us/step - loss: 106594.2925
Epoch 353/2000
1664/1664 [==============================] - 1s 480us/step - loss: 107229.5797
Epoch 354/2000
1664/1664 [==============================] - 1s 477us/step - loss: 102827.8446
Epoch 355/2000
1664/1664 [==============================] - 1s 481us/step -

...

1664/1664 [==============================] - 1s 467us/step - loss: 106195.9576
Epoch 1644/2000
1664/1664 [==============================] - 1s 477us/step - loss: 122920.8723
Epoch 1645/2000
1664/1664 [==============================] - 1s 463us/step - loss: 110845.0408
Epoch 1646/2000
1664/1664 [==============================] - 1s 475us/step - loss: 100532.3002
Epoch 1647/2000
1664/1664 [==============================] - 1s 473us/step - loss: 110968.9792
Epoch 1648/2000
1664/1664 [==============================] - 1s 483us/step - loss: 116473.5066
Epoch 1649/2000
1664/1664 [==============================] - 1s 469us/step - loss: 102824.3325
Epoch 1650/2000
1664/1664 [==============================] - 1s 480us/step - loss: 120891.7950
Epoch 1651/2000
1664/1664 [==============================] - 1s 479us/step - loss: 117551.0006
Epoch 1652/2000
1664/1664 [==============================] - 1s 476us/step - loss: 98994.7811
Epoch 1653/2000
1664/1664 [==============================] - 1s 479

1664/1664 [==============================] - 1s 475us/step - loss: 107588.5153
Epoch 1730/2000
1664/1664 [==============================] - 1s 468us/step - loss: 117530.4140
Epoch 1731/2000
1664/1664 [==============================] - 1s 475us/step - loss: 103110.2386
Epoch 1732/2000
1664/1664 [==============================] - 1s 476us/step - loss: 106759.2046
Epoch 1733/2000
1664/1664 [==============================] - 1s 474us/step - loss: 105126.6662
Epoch 1734/2000
1664/1664 [==============================] - 1s 468us/step - loss: 102217.9876
Epoch 1735/2000
1664/1664 [==============================] - 1s 476us/step - loss: 103003.2339
Epoch 1736/2000
1664/1664 [==============================] - 1s 473us/step - loss: 104284.1478
Epoch 1737/2000
1664/1664 [==============================] - 1s 467us/step - loss: 108725.7456
Epoch 1738/2000
1664/1664 [==============================] - 1s 477us/step - loss: 106610.6238
Epoch 1739/2000
1664/1664 [==============================] - 1s 45

1664/1664 [==============================] - 1s 479us/step - loss: 110485.4469
Epoch 1816/2000
1664/1664 [==============================] - 1s 473us/step - loss: 116386.7434
Epoch 1817/2000
1664/1664 [==============================] - 1s 464us/step - loss: 110577.9329
Epoch 1818/2000
1664/1664 [==============================] - 1s 477us/step - loss: 103980.2340
Epoch 1819/2000
1664/1664 [==============================] - 1s 466us/step - loss: 107739.4797
Epoch 1820/2000
1664/1664 [==============================] - 1s 475us/step - loss: 112293.2167
Epoch 1821/2000
1664/1664 [==============================] - 1s 470us/step - loss: 103690.4735
Epoch 1822/2000
1664/1664 [==============================] - 1s 477us/step - loss: 96498.9400
Epoch 1823/2000
1664/1664 [==============================] - 1s 476us/step - loss: 118453.8314
Epoch 1824/2000
1664/1664 [==============================] - 1s 462us/step - loss: 101097.8148
Epoch 1825/2000
1664/1664 [==============================] - 1s 473

1664/1664 [==============================] - 1s 473us/step - loss: 112050.4775
Epoch 1902/2000
1664/1664 [==============================] - 1s 467us/step - loss: 100032.4430
Epoch 1903/2000
1664/1664 [==============================] - 1s 482us/step - loss: 107757.6614
Epoch 1904/2000
1664/1664 [==============================] - 1s 466us/step - loss: 105279.0189
Epoch 1905/2000
1664/1664 [==============================] - 1s 478us/step - loss: 116109.0570
Epoch 1906/2000
1664/1664 [==============================] - 1s 470us/step - loss: 102180.8530
Epoch 1907/2000
1664/1664 [==============================] - 1s 472us/step - loss: 109380.7216
Epoch 1908/2000
1664/1664 [==============================] - 1s 463us/step - loss: 117209.6844
Epoch 1909/2000
1664/1664 [==============================] - 1s 475us/step - loss: 105883.9717
Epoch 1910/2000
1664/1664 [==============================] - 1s 474us/step - loss: 106736.0347
Epoch 1911/2000
1664/1664 [==============================] - 1s 48

1664/1664 [==============================] - 1s 477us/step - loss: 109171.4048
Epoch 1988/2000
1664/1664 [==============================] - 1s 468us/step - loss: 103640.2843
Epoch 1989/2000
1664/1664 [==============================] - 1s 495us/step - loss: 117205.8712
Epoch 1990/2000
1664/1664 [==============================] - 1s 488us/step - loss: 102143.3400
Epoch 1991/2000
1664/1664 [==============================] - 1s 473us/step - loss: 107699.4392
Epoch 1992/2000
1664/1664 [==============================] - 1s 473us/step - loss: 103464.8180
Epoch 1993/2000
1664/1664 [==============================] - 1s 475us/step - loss: 103988.9240
Epoch 1994/2000
1664/1664 [==============================] - 1s 490us/step - loss: 110886.7336
Epoch 1995/2000
1664/1664 [==============================] - 1s 476us/step - loss: 119770.3217
Epoch 1996/2000
1664/1664 [==============================] - 1s 474us/step - loss: 104230.74220s - lo
Epoch 1997/2000
1664/1664 [==============================] 